In [1]:
import os
import numpy as np
import numpy as np
# import torch
# from arsf_envi_reader import envi_header
import shutil
import os
import json
import math
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from osgeo import gdal,ogr,osr
from scipy.optimize import curve_fit
# from tqdm import tqdm
# import multiprocess as mp
from scipy import ndimage
from numpy import trapz
from scipy import stats
import rasterio

In [2]:
npy_folder = r'E:\wenqu\numpy_file\npy_file\site1b'  # where .npy files are stored
reference_tif = r'D:\wenqu\chapter1_2\updated_uas_modeling\uas_trait_map\site1b_ldmc.tif'    # an existing .tif file to copy georeferencing info


In [3]:
ref_ds = gdal.Open(reference_tif)
width = ref_ds.RasterXSize
height = ref_ds.RasterYSize
print(width, height)

10637 19347


# Graminoid

In [4]:
output_tif = r'D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_graminoid.tif'


# Your list of selected bands
band_names = ['b55_std', 'b72_mean', 'b62_std', 'b30_mean', 'b42_mean',
       'b63_mean', 'b117_std', 'b113_std', 'b11_std', 'b90_mean',
       'b1_mean', 'b82_std', 'b2_mean', 'b8_mean', 'b99_mean', 'b66_std',
       'b103_mean', 'b68_std', 'b105_mean', 'b18_std', 'b49_std',
       'b43_std', 'b49_mean', 'b26_std', 'b109_std', 'b33_mean',
       'b46_std', 'b5_std', 'b20_mean', 'b91_std', 'b13_mean', 'b97_std',
       'b102_std', 'b19_mean', 'b7_std', 'b22_std', 'b22_mean', 'b2_std',
       'b91_mean', 'b35_std', 'b24_mean', 'b9_mean', 'b96_std', 'b33_std',
       'b37_std', 'b104_mean', 'b15_mean', 'b111_std', 'b122_std',
       'b121_std', 'b113_mean', 'b99_std', 'b108_mean', 'b119_mean',
       'b26_mean', 'b28_std', 'b110_mean', 'b110_std', 'b119_std',
       'b74_mean', 'b59_std', 'b11_mean', 'b115_mean', 'b17_std',
       'b50_std', 'b98_std', 'b109_mean', 'b5_mean', 'b106_std',
       'b13_std', 'b21_mean', 'b104_std', 'b107_mean', 'b111_mean',
       'b4_std', 'b6_std', 'b29_std', 'b107_std', 'b8_std', 'b3_std',
       'b117_mean', 'b120_std', 'b121_mean', 'b118_std']


plsr_coefficients  = np.array([27.617078410311645,
 -0.38344441784305366,
 -14.848183274386573,
 6.932289772131841,
 0.10861065838261054,
 2.9691436797025785,
 -4.406512745651725,
 -3.6509987163526296,
 3.198719702656972,
 -0.7191742541098439,
 -3.207903165722437,
 5.969197501581659,
 1.2039256583669238,
 13.630245820268469,
 -1.2283606952171249,
 -4.582672242503678,
 -1.2542542849921356,
 -0.7632283474054853,
 -1.4406715606619798,
 -34.368377779191526,
 13.432874373657603,
 7.823175200868005,
 3.5785850701448845,
 14.074775894146542,
 -2.1128347372361134,
 8.906879544826158,
 -16.235494253772725,
 -30.601639017737874,
 -4.7611832612703004,
 -0.48431603765563036,
 -9.118658559440586,
 -1.402018072871836,
 1.8531077249980306,
 -7.761114838547131,
 17.711648025793554,
 -2.8535428415870894,
 -13.024661833205096,
 -8.35254681428451,
 -1.510338472299643,
 -17.419931679031762,
 -9.256833839302159,
 2.301637187507047,
 -1.8794760419978838,
 -15.87215006016524,
 7.803478953070631,
 -1.2419310783283768,
 -12.963046331998594,
 4.613494915982251,
 4.390025405380304,
 2.774024357284024,
 -1.9042343613106585,
 -4.388013466368747,
 0.9716650156721711,
 1.2653033110388276,
 10.907248995852612,
 -22.479683548283457,
 1.9223411238951824,
 5.4057002267753225,
 -2.732393531305007,
 1.5550564746114675,
 -19.417801933078064,
 21.1226259245573,
 -1.1256629567862304,
 -42.45612179819905,
 12.129784581585747,
 5.79292177726388,
 1.8529844070165034,
 27.34523024680145,
 7.304225725425076,
 39.38834139231638,
 -15.235336499849756,
 -3.6548411494583615,
 1.5834495720552932,
 -1.9013801421036536,
 -32.945446940160465,
 53.18732323104778,
 44.19273314545867,
 5.06485265924453,
 57.159051748157836,
 -37.143897093567325,
 -2.23905418527548,
 9.9845368822406,
 3.628699963943002,
 -22.479877963591385])

intercept = 0.43885487
trait_map = None


# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")


Successfully created trait map: D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_graminoid.tif
Trait map stats - Min: -10.04, Max: 1.54, Mean: -1.23


# lichen

# scale

In [4]:
output_tif = r'D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_lichen.tif'


# Your list of selected bands
band_names = ['b115_std', 'b33_mean', 'b68_std', 'b19_std', 'b47_mean',
       'b83_std', 'b48_mean', 'b103_mean', 'b99_mean', 'b96_std',
       'b28_std', 'b23_std', 'b64_std', 'b25_mean', 'b14_mean',
       'b18_mean', 'b98_std', 'b50_std', 'b15_mean', 'b115_mean',
       'b56_mean', 'b57_std', 'b63_std', 'b121_mean', 'b46_mean',
       'b27_std', 'b108_mean', 'b37_std', 'b20_mean', 'b15_std',
       'b117_std', 'b46_std', 'b121_std', 'b110_std', 'b111_std',
       'b122_std', 'b58_std', 'b24_std', 'b119_mean', 'b111_mean',
       'b57_mean', 'b114_std', 'b13_std', 'b5_mean', 'b103_std',
       'b109_std', 'b55_mean', 'b112_std', 'b116_std', 'b26_std',
       'b104_std', 'b120_std', 'b117_mean', 'b107_std', 'b17_std',
       'b34_std', 'b13_mean', 'b31_std', 'b118_std', 'b26_mean',
       'b109_mean', 'b5_std', 'b25_std', 'b32_std', 'b118_mean', 'b2_std',
       'b59_std', 'b56_std', 'b6_mean', 'b18_std', 'b85_std', 'b6_std',
       'b8_mean', 'b119_std']


plsr_coefficients  = np.array([-9.12848423571234,
 -23.468696289947648,
 -1.9602772975773315,
 -35.46462953213529,
 12.158769855205186,
 -0.6162595449615543,
 18.87618882370926,
 -0.1592221916257049,
 0.10366283250958973,
 -4.533535113352133,
 45.79035386149358,
 25.81341628159682,
 -11.917719513656355,
 -13.582629312390916,
 28.19618232552195,
 -3.86881776293363,
 3.2474308242928416,
 -23.624272631479585,
 13.266957053364141,
 3.459184158284762,
 -12.917931782902349,
 -35.82738394151332,
 -11.430249875238243,
 -3.411902527245407,
 16.88967912180593,
 -21.64743858557482,
 -1.0415597005417352,
 -35.6107369097379,
 -0.413673455198777,
 -33.278081947198345,
 -2.1961465510118128,
 -3.15832401902157,
 10.910445442173653,
 -6.645267827882973,
 -9.746228980698778,
 5.952301517601169,
 -22.353426101497107,
 -12.30369008766176,
 -0.8503077699403198,
 1.5503545772965504,
 -11.848603361109657,
 8.392943264498864,
 -76.30019483065672,
 -33.91641224467068,
 8.14181763864398,
 -5.030596144983438,
 -14.906082518031535,
 -3.2053706169339664,
 -2.225785319946753,
 -58.655570084641106,
 7.278366229570919,
 -11.190693665380637,
 2.8744292964062432,
 -1.505295975405057,
 62.217049338795356,
 51.1131960049503,
 35.242392426509106,
 66.28540885583402,
 7.876691559127418,
 -12.086066679031214,
 -3.9108256066163496,
 81.8760091989582,
 -111.5138222451565,
 25.45022452167932,
 3.2008042064192495,
 54.77914779498349,
 42.92921465479339,
 76.58278311950099,
 -77.89176288157778,
 62.88109581086722,
 -10.229122271098182,
 -99.31787300720437,
 46.296830849113526,
 15.42221335040406]
)

intercept = - 0.03521068
trait_map = None


# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")


Successfully created trait map: D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_lichen.tif
Trait map stats - Min: -1.55, Max: 11.26, Mean: 1.43


# Forbs

# scale

In [4]:
output_tif = r'D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_forbs.tif'


# # Your list of selected bands
band_names = ['b15_mean', 'b50_mean', 'b59_mean', 'b1_mean', 'b6_mean',
       'b114_mean', 'b74_std', 'b116_mean', 'b51_mean', 'b104_std',
       'b120_mean', 'b32_mean', 'b106_mean', 'b109_std', 'b25_std',
       'b121_mean', 'b105_mean', 'b116_std', 'b4_mean', 'b9_mean',
       'b16_std', 'b29_std', 'b8_std', 'b120_std', 'b14_mean', 'b2_mean',
       'b17_std']

plsr_coefficients  = np.array([26.537352223678653,
 2.0363826828202223,
 -11.923345291574671,
 2.573041592162637,
 14.143751335990531,
 -1.2528757745241375,
 -1.1193775187808355,
 -3.4382668213455205,
 17.35538191243584,
 -1.6268239957412793,
 -0.8051610658646301,
 -6.37921345482175,
 0.18643051147787823,
 -0.6369334982253737,
 -36.89549982753157,
 1.5599231150426944,
 4.002828280941294,
 -0.4859903790756567,
 -11.744572398381838,
 -12.259411622262606,
 -35.35698227333474,
 18.938729569584435,
 14.180245869193294,
 3.5332639214738877,
 -27.913646074377333,
 1.3827268808909174,
 48.19397605686376])


intercept = 0.02778703
trait_map = None
# Loop through bands and apply coefficients
for band_name, coef in zip(band_names, plsr_coefficients):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    if not os.path.exists(band_path):
        print(f"Missing file: {band_path}")
        continue
    band_array = np.load(band_path)

    if trait_map is None:
        trait_map = np.zeros_like(band_array)

    trait_map += band_array * coef

# Add intercept if available
trait_map += intercept

In [5]:
# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

Successfully created trait map: D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_forbs.tif
Trait map stats - Min: -0.67, Max: 21.05, Mean: 2.82


# evergreen_shrub

# scale

In [11]:
output_tif = r'D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_evergreen_shrub.tif'


# Your list of selected bands
band_names = ['b14_std', 'b1_std', 'b65_std', 'b55_std', 'b15_mean', 'b58_std',
       'b56_std', 'b19_mean', 'b57_std', 'b18_mean', 'b20_mean',
       'b13_mean', 'b7_mean', 'b60_std', 'b118_std', 'b64_std', 'b63_std',
       'b62_std', 'b61_std', 'b3_mean', 'b6_mean', 'b4_mean', 'b5_mean',
       'b2_mean']

plsr_coefficients  = np.array([3.8395576811953247,
 0.26085904474892435,
 -0.25982156764050135,
 -0.5214775038905953,
 1.6194913438931478,
 -0.4794912047594231,
 -0.6177121105172235,
 1.6280086909490037,
 -0.46991307401697113,
 1.7314793682588951,
 1.6954193605685077,
 2.285099718229991,
 -0.49052925659078106,
 -0.3862659555990946,
 0.9968830111355832,
 -0.4000796497332469,
 -0.4963696084412918,
 -0.6283045207819942,
 -0.8906833652683365,
 -1.0498281792997517,
 -2.092078150764214,
 -1.2800537107862306,
 -2.512106685337876,
 -1.7132077369143057])

intercept = -0.04467753
trait_map = None

# Loop through bands and apply coefficients
for band_name, coef in zip(band_names, plsr_coefficients):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    if not os.path.exists(band_path):
        print(f"Missing file: {band_path}")
        continue
    band_array = np.load(band_path)

    if trait_map is None:
        trait_map = np.zeros_like(band_array)

    trait_map += band_array * coef

# Add intercept if available
trait_map += intercept


In [12]:
# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

Successfully created trait map: D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_evergreen_shrub.tif
Trait map stats - Min: -0.13, Max: 1.37, Mean: 0.19


# deciduous_shrub

# scale

In [10]:
output_tif = r'D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_deciduous_shrub.tif'


# Your list of selected bands
band_names = ['b22_std', 'b65_std', 'b109_std', 'b111_std', 'b38_std', 'b2_mean',
       'b50_std', 'b13_std', 'b119_std', 'b96_std', 'b90_std', 'b89_std',
       'b107_std', 'b85_mean', 'b14_std', 'b114_mean', 'b91_mean',
       'b121_mean', 'b60_mean', 'b118_mean', 'b92_std', 'b60_std',
       'b82_std', 'b27_std', 'b3_std', 'b28_mean', 'b57_std', 'b98_mean',
       'b37_mean', 'b110_mean', 'b95_std', 'b94_std', 'b37_std',
       'b116_mean', 'b16_std', 'b7_std', 'b118_std', 'b15_std',
       'b74_mean', 'b7_mean', 'b32_std', 'b113_mean', 'b34_std',
       'b100_std', 'b59_std', 'b108_mean', 'b20_mean', 'b6_mean',
       'b17_std', 'b8_mean', 'b98_std']

plsr_coefficients  = np.array([98.04665027211357,
 1.0708815176971,
 12.36562310752248,
 -1.5724366236405578,
 -120.78883742900173,
 -12.92119652055286,
 85.65324553137312,
 -42.775103768599855,
 -3.7745821538061723,
 0.03959261631359633,
 16.461709760645064,
 -10.925995251217573,
 12.111272900781074,
 3.2290867441432085,
 -40.73480125471686,
 -7.445563421289272,
 2.56221126838098,
 -7.508956859889747,
 3.5000349803877278,
 2.874118429346,
 -9.63473852591215,
 46.984469206643425,
 21.30485236603342,
 80.10278588722821,
 17.16510706410676,
 4.994141710750532,
 32.17743464384951,
 -6.7043808627598915,
 -20.86056629090262,
 4.137079479742989,
 -1.4350453328312602,
 17.13827305511386,
 112.48766627878815,
 -4.9816377914617975,
 95.23054620311623,
 -37.57941121067769,
 7.824489517254509,
 9.529790728072944,
 1.2160357492260643,
 -21.057460009446217,
 -76.0817279021665,
 6.977762900599133,
 -6.530761316898698,
 -21.002005501713754,
 -122.02958366666992,
 6.495085743273609,
 48.42429548242809,
 95.59669156705694,
 -148.80322106121776,
 -110.15929883983134,
 -32.36763480843152])

intercept =-0.26072803
trait_map = None

# Loop through bands and apply coefficients
for band_name, coef in zip(band_names, plsr_coefficients):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    if not os.path.exists(band_path):
        print(f"Missing file: {band_path}")
        continue
    band_array = np.load(band_path)

    if trait_map is None:
        trait_map = np.zeros_like(band_array)

    trait_map += band_array * coef

# Add intercept if available
trait_map += intercept

# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

Successfully created trait map: D:\wenqu\chapter1_2\chapter2\UAS_PFT_Update1\site1b_deciduous_shrub.tif
Trait map stats - Min: -2.20, Max: 1.81, Mean: 0.16


# d13c

# scale

In [4]:
output_tif = r'D:\wenqu\chapter1_2\uas_trait_map\site6_d13c.tif'


# Your list of selected bands
band_names = ['b82_mean', 'b56_mean', 'b75_mean', 'b67_mean', 'b99_mean',
       'b97_mean', 'b57_mean', 'b35_mean', 'b26_mean', 'b5_mean',
       'b116_mean', 'b17_mean', 'b1_mean', 'b14_mean', 'b73_mean',
       'b28_mean', 'b3_mean', 'b39_mean', 'b51_mean', 'b122_mean',
       'b40_mean', 'b30_mean', 'b49_mean', 'b42_mean', 'b76_mean',
       'b48_mean', 'b8_mean', 'b34_mean', 'b121_mean', 'b52_mean',
       'b15_mean', 'b7_mean']

plsr_coefficients  = np.array([236.61852623486863,
 -482.88222925347776,
 138.77216745347565,
 -44.673199569661506,
 -130.83854381839146,
 -69.32322347897814,
 -130.99011762271542,
 174.92483319157319,
 86.31135633974849,
 103.09103836913387,
 44.41173006786578,
 -0.13247107610119974,
 83.19520249834109,
 525.2716626984129,
 78.22438490255212,
 290.4246751239607,
 -97.49652904815359,
 -232.4310991099464,
 161.20342046086108,
 104.632806150325,
 -387.46969869396503,
 116.29419474030463,
 138.606982935393,
 -116.52149104887903,
 -357.40088662930526,
 -128.67990761302707,
 -366.92702046262247,
 -364.4010164924634,
 -70.19222226778966,
 765.1253660579733,
 458.31738032251434,
 -476.77803435807124])

intercept = -27.40784303
trait_map = None

# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

AttributeError: 'NoneType' object has no attribute 'SetGeoTransform'

# d15n

In [48]:
output_tif = r'D:\wenqu\chapter1_2\uas_trait_map\site2c_d15n.tif'


# PLSR model parameters
band_names = ['b74_mean', 'b5_mean', 'b57_mean', 'b58_mean', 'b38_std',
             'b60_mean', 'b25_std', 'b95_std', 'b30_std', 'b9_mean', 'b114_std',
             'b91_std', 'b23_std', 'b18_std', 'b111_mean', 'b14_mean',
             'b101_std', 'b7_mean', 'b122_std', 'b59_std', 'b107_mean',
             'b118_mean', 'b122_mean', 'b13_std', 'b10_std', 'b98_std',
             'b3_mean', 'b99_std', 'b15_std', 'b109_std', 'b14_std', 'b9_std',
             'b8_mean', 'b108_mean', 'b113_std', 'b118_std']

plsr_coefficients = np.array([61.95069832038617,
 -443.8175861956897,
 244.61697666292002,
 161.76757240677037,
 890.6348221654105,
 -179.8869698051848,
 -510.1111454297827,
 149.63785994193933,
 -353.6891690306521,
 -537.7722892932843,
 111.57768804560136,
 -24.366765168206275,
 262.9394749098357,
 -726.5451610814567,
 -22.324825795701166,
 -640.9355498468841,
 -41.70931448381327,
 1007.2321141650461,
 77.62091383540057,
 -535.1730005567188,
 64.55454750642573,
 -69.0054026100846,
 -48.2102651040233,
 282.5904906929578,
 -1322.7513820446086,
 16.7844528529304,
 -157.0443501507085,
 -279.05763846899987,
 -584.5370647143357,
 199.05976457311684,
 1960.7291117127768,
 1359.530026185485,
 616.2063977631504,
 30.11636147145316,
 -182.94787076813563,
 -23.993009150288234])

intercept = -4.16656468

# Initialize trait map
trait_map = None
first_band = True

# Load reference image for size and geo info
ref_ds = gdal.Open(reference_tif)
geotransform = ref_ds.GetGeoTransform()
projection = ref_ds.GetProjection()
cols = ref_ds.RasterXSize
rows = ref_ds.RasterYSize
ref_ds = None

# Initialize trait map and background mask
trait_map = np.zeros((rows, cols), dtype=np.float32)
background_mask = None

# Process each band
for idx, (band_name, coef) in enumerate(zip(band_names, plsr_coefficients)):
    band_path = os.path.join(npy_folder, f"{band_name}.npy")
    
    if not os.path.exists(band_path):
        print(f"Warning: Missing band file {band_path}")
        continue
    
    try:
        band_array = np.load(band_path)
        
        # Check dimension match
        if band_array.shape != (rows, cols):
            print(f"Warning: {band_name} has wrong dimensions {band_array.shape}, expected {(rows, cols)}")
            continue
        
        # Create background mask from the FIRST band
        if idx == 0:
            background_mask = (band_array == 0)  # True = background
        
        # Multiply only valid data
        trait_map += band_array * coef
    
    except Exception as e:
        print(f"Error processing {band_name}: {str(e)}")
        continue

# Add intercept
trait_map += intercept

# Mask background as NaN
trait_map = np.where(background_mask, 0, trait_map)

# Write output GeoTIFF
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create(output_tif, cols, rows, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(geotransform)
out_ds.SetProjection(projection)
out_ds.GetRasterBand(1).WriteArray(trait_map)
out_ds.FlushCache()
out_ds = None

print(f"Successfully created trait map: {output_tif}")
print(f"Trait map stats - Min: {np.nanmin(trait_map):.2f}, Max: {np.nanmax(trait_map):.2f}, Mean: {np.nanmean(trait_map):.2f}")

Successfully created trait map: D:\wenqu\chapter1_2\uas_trait_map\site2c_d15n.tif
Trait map stats - Min: -23.05, Max: 22.20, Mean: 0.40
